In [3]:
import numpy as np
import pandas as pd

In [4]:
dat = pd.read_csv("bgg_dataset_raw.csv", delimiter=";").dropna().reset_index()
dat.head()

,index,ID,Name,Year Published,Min Players,Max Players,Play Time,Min Age,Users Rated,Rating Average,BGG Rank,Complexity Average,Owned Users,Mechanics,Domains
0,0,174430.0,Gloomhaven,2017.0,1,4,120,14,42055,"8,79",1,"3,86",68323.0,"Action Queue, Action Retrieval, Campaign / Bat...","Strategy Games, Thematic Games"
1,1,161936.0,Pandemic Legacy: Season 1,2015.0,2,4,60,13,41643,"8,61",2,"2,84",65294.0,"Action Points, Cooperative Game, Hand Manageme...","Strategy Games, Thematic Games"
2,2,224517.0,Brass: Birmingham,2018.0,2,4,120,14,19217,"8,66",3,"3,91",28785.0,"Hand Management, Income, Loans, Market, Networ...",Strategy Games
3,3,167791.0,Terraforming Mars,2016.0,1,5,120,12,64864,"8,43",4,"3,24",87099.0,"Card Drafting, Drafting, End Game Bonuses, Han...",Strategy Games
4,4,233078.0,Twilight Imperium: Fourth Edition,2017.0,3,6,480,14,13468,"8,70",5,"4,22",16831.0,"Action Drafting, Area Majority / Influence, Ar...","Strategy Games, Thematic Games"


In [5]:
dat.columns = dat.columns.str.lower().str.replace(" ", "_", regex=True)
dat = dat.astype({
    "id": "int64",
    "year_published": "int64",
    "owned_users": "int64",
})
dat["rating_average"] = dat["rating_average"].str.replace(",", ".").astype(float)
dat["complexity_average"] = dat["complexity_average"].str.replace(",", ".").astype(float)

mechanics_split = dat["mechanics"].str.split(", ", expand=True)
mechanics_split.columns = [f"mechanics{i+1}" for i in range(mechanics_split.shape[1])]

domains_split = dat["domains"].str.split(", ", expand=True)
domains_split.columns = [f"domains{i+1}" for i in range(domains_split.shape[1])]

dat = pd.concat([dat, mechanics_split, domains_split], axis=1)

In [6]:
dat.head()

,index,id,name,year_published,min_players,max_players,play_time,min_age,users_rated,rating_average,...,mechanics13,mechanics14,mechanics15,mechanics16,mechanics17,mechanics18,mechanics19,domains1,domains2,domains3
0,0,174430,Gloomhaven,2017,1,4,120,14,42055,8.79,...,Modular Board,Once-Per-Game Abilities,Scenario / Mission / Campaign Game,Simultaneous Action Selection,Solo / Solitaire Game,Storytelling,Variable Player Powers,Strategy Games,Thematic Games,None
1,1,161936,Pandemic Legacy: Season 1,2015,2,4,60,13,41643,8.61,...,None,None,None,None,None,None,None,Strategy Games,Thematic Games,None
2,2,224517,Brass: Birmingham,2018,2,4,120,14,19217,8.66,...,None,None,None,None,None,None,None,Strategy Games,None,None
3,3,167791,Terraforming Mars,2016,1,5,120,12,64864,8.43,...,None,None,None,None,None,None,None,Strategy Games,None,None
4,4,233078,Twilight Imperium: Fourth Edition,2017,3,6,480,14,13468,8.70,...,None,None,None,None,None,None,None,Strategy Games,Thematic Games,None


In [7]:
dat.columns

Index(['index', 'id', 'name', 'year_published', 'min_players', 'max_players',
       'play_time', 'min_age', 'users_rated', 'rating_average', 'bgg_rank',
       'complexity_average', 'owned_users', 'mechanics', 'domains',
       'mechanics1', 'mechanics2', 'mechanics3', 'mechanics4', 'mechanics5',
       'mechanics6', 'mechanics7', 'mechanics8', 'mechanics9', 'mechanics10',
       'mechanics11', 'mechanics12', 'mechanics13', 'mechanics14',
       'mechanics15', 'mechanics16', 'mechanics17', 'mechanics18',
       'mechanics19', 'domains1', 'domains2', 'domains3'],
      dtype='object')

In [10]:
dat = dat[dat["max_players"] <= 30]

In [11]:
print(dat)

      index      id                               name  year_published  \
0         0  174430                         Gloomhaven            2017   
1         1  161936          Pandemic Legacy: Season 1            2015   
2         2  224517                  Brass: Birmingham            2018   
3         3  167791                  Terraforming Mars            2016   
4         4  233078  Twilight Imperium: Fourth Edition            2017   
...     ...     ...                                ...             ...   
9703  20336    2921                   The Game of Life            1960   
9704  20337    1410                            Trouble            1965   
9706  20340    5048                         Candy Land            1949   
9707  20341    5432                 Chutes and Ladders            -200   
9708  20342   11901                        Tic-Tac-Toe           -1300   

      min_players  max_players  play_time  min_age  users_rated  \
0               1            4        120   

In [13]:
dat = dat[dat["max_players"] != 0]

In [14]:
print(dat)

      index      id                               name  year_published  \
0         0  174430                         Gloomhaven            2017   
1         1  161936          Pandemic Legacy: Season 1            2015   
2         2  224517                  Brass: Birmingham            2018   
3         3  167791                  Terraforming Mars            2016   
4         4  233078  Twilight Imperium: Fourth Edition            2017   
...     ...     ...                                ...             ...   
9703  20336    2921                   The Game of Life            1960   
9704  20337    1410                            Trouble            1965   
9706  20340    5048                         Candy Land            1949   
9707  20341    5432                 Chutes and Ladders            -200   
9708  20342   11901                        Tic-Tac-Toe           -1300   

      min_players  max_players  play_time  min_age  users_rated  \
0               1            4        120   

In [22]:
dat = dat[dat["name"].str.count("-") <= 1]

In [23]:
print(dat)

      index      id                               name  year_published  \
0         0  174430                         Gloomhaven            2017   
1         1  161936          Pandemic Legacy: Season 1            2015   
2         2  224517                  Brass: Birmingham            2018   
3         3  167791                  Terraforming Mars            2016   
4         4  233078  Twilight Imperium: Fourth Edition            2017   
...     ...     ...                                ...             ...   
9702  20335    1406                           Monopoly            1933   
9703  20336    2921                   The Game of Life            1960   
9704  20337    1410                            Trouble            1965   
9706  20340    5048                         Candy Land            1949   
9707  20341    5432                 Chutes and Ladders            -200   

      min_players  max_players  play_time  min_age  users_rated  \
0               1            4        120   

In [24]:
dat = dat[dat["name"].str.count("&") <= 1]

In [25]:
print(dat)

      index      id                               name  year_published  \
0         0  174430                         Gloomhaven            2017   
1         1  161936          Pandemic Legacy: Season 1            2015   
2         2  224517                  Brass: Birmingham            2018   
3         3  167791                  Terraforming Mars            2016   
4         4  233078  Twilight Imperium: Fourth Edition            2017   
...     ...     ...                                ...             ...   
9702  20335    1406                           Monopoly            1933   
9703  20336    2921                   The Game of Life            1960   
9704  20337    1410                            Trouble            1965   
9706  20340    5048                         Candy Land            1949   
9707  20341    5432                 Chutes and Ladders            -200   

      min_players  max_players  play_time  min_age  users_rated  \
0               1            4        120   

In [26]:
dat = dat[dat["rating_average"]>=1.5]

In [27]:
print(dat)

      index      id                               name  year_published  \
0         0  174430                         Gloomhaven            2017   
1         1  161936          Pandemic Legacy: Season 1            2015   
2         2  224517                  Brass: Birmingham            2018   
3         3  167791                  Terraforming Mars            2016   
4         4  233078  Twilight Imperium: Fourth Edition            2017   
...     ...     ...                                ...             ...   
9702  20335    1406                           Monopoly            1933   
9703  20336    2921                   The Game of Life            1960   
9704  20337    1410                            Trouble            1965   
9706  20340    5048                         Candy Land            1949   
9707  20341    5432                 Chutes and Ladders            -200   

      min_players  max_players  play_time  min_age  users_rated  \
0               1            4        120   

In [28]:
dat.to_json("bgg_dataset_cleaned.json", orient="records", indent=4)